In [23]:
import tensorflow as tf
import keras
from keras.layers import Input, Dense, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import os
import h5py
import pandas as pd
import numpy as np
from jarvis.db.figshare import data
import csv


In [24]:
df1 = pd.read_csv('data/train.csv')
other = pd.read_csv('data/unique_m.csv')
df = pd.concat([df1,other],axis=1)
original_columns = len(df.columns)
print(df.shape)

(21263, 170)


In [25]:
df = df.T.drop_duplicates().T

In [26]:
df = df.drop('material', axis = 1)

In [27]:
for x in df.columns:
    print(x)

number_of_elements
mean_atomic_mass
wtd_mean_atomic_mass
gmean_atomic_mass
wtd_gmean_atomic_mass
entropy_atomic_mass
wtd_entropy_atomic_mass
range_atomic_mass
wtd_range_atomic_mass
std_atomic_mass
wtd_std_atomic_mass
mean_fie
wtd_mean_fie
gmean_fie
wtd_gmean_fie
entropy_fie
wtd_entropy_fie
range_fie
wtd_range_fie
std_fie
wtd_std_fie
mean_atomic_radius
wtd_mean_atomic_radius
gmean_atomic_radius
wtd_gmean_atomic_radius
entropy_atomic_radius
wtd_entropy_atomic_radius
range_atomic_radius
wtd_range_atomic_radius
std_atomic_radius
wtd_std_atomic_radius
mean_Density
wtd_mean_Density
gmean_Density
wtd_gmean_Density
entropy_Density
wtd_entropy_Density
range_Density
wtd_range_Density
std_Density
wtd_std_Density
mean_ElectronAffinity
wtd_mean_ElectronAffinity
gmean_ElectronAffinity
wtd_gmean_ElectronAffinity
entropy_ElectronAffinity
wtd_entropy_ElectronAffinity
range_ElectronAffinity
wtd_range_ElectronAffinity
std_ElectronAffinity
wtd_std_ElectronAffinity
mean_FusionHeat
wtd_mean_FusionHeat
gmean

In [28]:
df = df.drop(df.columns[[0,1,2,3,4,5,6,7,8,9,10,11]], axis=1)

In [29]:
tc = df['critical_temp']

In [30]:
df = df.loc[:,'H':'Bi']
df

,H,He,Li,Be,B,C,N,O,F,Na,...,W,Re,Os,Ir,Pt,Au,Hg,Tl,Pb,Bi
0,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21258,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21259,0.0,0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
21260,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21261,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
#df = df[df.critical_temp >25]

In [32]:
#df = df[df.critical_temp != 0]

In [33]:
df['critical_temp']

KeyError: 'critical_temp'

In [36]:
X_train, X_test, y_train, y_test = train_test_split(df, tc, test_size=0.2, random_state=42)


In [37]:
X_train.shape

(17010, 78)

In [42]:
Inputs = Input(shape=(78))
x = BatchNormalization()(Inputs)

x = Dense(256, activation='relu', kernel_initializer='lecun_uniform', name='fc1_relu')(x)
x = Dense(128, activation='relu', kernel_initializer='lecun_uniform', name = 'fc2')(x)

x = Dense(64, activation='relu', kernel_initializer='lecun_uniform', name = 'fc3')(x)
x = Dense(32, activation='relu', kernel_initializer='lecun_uniform', name='fc4')(x)
predictions = Dense(1, activation='relu', kernel_initializer='lecun_uniform', name = 'relu')(x)
model = Model(inputs=Inputs, outputs=predictions)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 78)]              0         
                                                                 
 batch_normalization_1 (Batc  (None, 78)               312       
 hNormalization)                                                 
                                                                 
 fc1_relu (Dense)            (None, 256)               20224     
                                                                 
 fc2 (Dense)                 (None, 128)               32896     
                                                                 
 fc3 (Dense)                 (None, 64)                8256      
                                                                 
 fc4 (Dense)                 (None, 32)                2080      
                                                           

In [43]:
adam = Adam(lr=0.0001)
model.compile(optimizer=adam, loss='MSE')

In [44]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)


In [45]:
history = model.fit(np.asarray(X_train).astype('float32'), np.asarray(y_train).astype('float32'), batch_size = 1024, epochs = 1000, 
                    validation_split = 0.25, shuffle = True, callbacks = None,
                    use_multiprocessing=True, workers=4)

Epoch 1/1000


2023-02-24 02:05:53.971984: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


13/13 [==============================] - 2s 47ms/step - loss: 2363.6416 - val_loss: 2380.6477
Epoch 2/1000
 8/13 [=================>............] - ETA: 0s - loss: 2354.1389

2023-02-24 02:05:56.154005: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


13/13 [==============================] - 0s 10ms/step - loss: 2348.2415 - val_loss: 2363.9602
Epoch 3/1000
13/13 [==============================] - 0s 9ms/step - loss: 2326.2764 - val_loss: 2348.8745
Epoch 4/1000
13/13 [==============================] - 0s 9ms/step - loss: 2300.3752 - val_loss: 2330.5723
Epoch 5/1000
13/13 [==============================] - 0s 8ms/step - loss: 2267.5706 - val_loss: 2306.1934
Epoch 6/1000
13/13 [==============================] - 0s 8ms/step - loss: 2224.1230 - val_loss: 2273.5325
Epoch 7/1000
13/13 [==============================] - 0s 8ms/step - loss: 2167.0034 - val_loss: 2229.0059
Epoch 8/1000
13/13 [==============================] - 0s 8ms/step - loss: 2090.9949 - val_loss: 2168.1150
Epoch 9/1000
13/13 [==============================] - 0s 8ms/step - loss: 1992.3171 - val_loss: 2088.5491
Epoch 10/1000
13/13 [==============================] - 0s 8ms/step - loss: 1871.9310 - val_loss: 1986.7924
Epoch 11/1000
13/13 [==============================] - 0s

...

13/13 [==============================] - 0s 8ms/step - loss: 157.4162 - val_loss: 192.1152
Epoch 620/1000
13/13 [==============================] - 0s 9ms/step - loss: 156.9825 - val_loss: 198.1111
Epoch 621/1000
13/13 [==============================] - 0s 8ms/step - loss: 155.0519 - val_loss: 193.1172
Epoch 622/1000
13/13 [==============================] - 0s 8ms/step - loss: 154.8439 - val_loss: 193.3439
Epoch 623/1000
13/13 [==============================] - 0s 8ms/step - loss: 154.8381 - val_loss: 200.0992
Epoch 624/1000
13/13 [==============================] - 0s 8ms/step - loss: 156.9672 - val_loss: 194.5453
Epoch 625/1000
13/13 [==============================] - 0s 8ms/step - loss: 155.7089 - val_loss: 191.4333
Epoch 626/1000
13/13 [==============================] - 0s 8ms/step - loss: 154.0607 - val_loss: 197.6010
Epoch 627/1000
13/13 [==============================] - 0s 8ms/step - loss: 154.7221 - val_loss: 190.4984
Epoch 628/1000
13/13 [==============================] - 0s 8m

13/13 [==============================] - 0s 9ms/step - loss: 153.7719 - val_loss: 195.2397
Epoch 697/1000
13/13 [==============================] - 0s 9ms/step - loss: 149.6504 - val_loss: 196.9046
Epoch 698/1000
13/13 [==============================] - 0s 8ms/step - loss: 150.3544 - val_loss: 195.3272
Epoch 699/1000
13/13 [==============================] - 0s 9ms/step - loss: 153.0409 - val_loss: 189.9336
Epoch 700/1000
13/13 [==============================] - 0s 9ms/step - loss: 150.2617 - val_loss: 197.2868
Epoch 701/1000
13/13 [==============================] - 0s 8ms/step - loss: 153.5707 - val_loss: 189.6350
Epoch 702/1000
13/13 [==============================] - 0s 8ms/step - loss: 150.5538 - val_loss: 191.5629
Epoch 703/1000
13/13 [==============================] - 0s 9ms/step - loss: 154.3640 - val_loss: 191.1705
Epoch 704/1000
13/13 [==============================] - 0s 8ms/step - loss: 150.7784 - val_loss: 187.4695
Epoch 705/1000
13/13 [==============================] - 0s 8m

13/13 [==============================] - 0s 8ms/step - loss: 149.8889 - val_loss: 187.5145
Epoch 774/1000
13/13 [==============================] - 0s 8ms/step - loss: 148.8086 - val_loss: 188.5750
Epoch 775/1000
13/13 [==============================] - 0s 8ms/step - loss: 145.7806 - val_loss: 187.1834
Epoch 776/1000
13/13 [==============================] - 0s 8ms/step - loss: 149.8514 - val_loss: 187.1780
Epoch 777/1000
13/13 [==============================] - 0s 8ms/step - loss: 148.3915 - val_loss: 188.8216
Epoch 778/1000
13/13 [==============================] - 0s 8ms/step - loss: 149.0239 - val_loss: 190.1469
Epoch 779/1000
13/13 [==============================] - 0s 8ms/step - loss: 150.9518 - val_loss: 203.9802
Epoch 780/1000
13/13 [==============================] - 0s 7ms/step - loss: 150.2500 - val_loss: 195.1557
Epoch 781/1000
13/13 [==============================] - 0s 7ms/step - loss: 149.5891 - val_loss: 186.3367
Epoch 782/1000
13/13 [==============================] - 0s 7m

13/13 [==============================] - 0s 8ms/step - loss: 154.2228 - val_loss: 184.3932
Epoch 851/1000
13/13 [==============================] - 0s 8ms/step - loss: 148.6753 - val_loss: 192.0317
Epoch 852/1000
13/13 [==============================] - 0s 8ms/step - loss: 146.5186 - val_loss: 197.3560
Epoch 853/1000
13/13 [==============================] - 0s 8ms/step - loss: 148.0715 - val_loss: 185.3327
Epoch 854/1000
13/13 [==============================] - 0s 8ms/step - loss: 152.4392 - val_loss: 185.0558
Epoch 855/1000
13/13 [==============================] - 0s 8ms/step - loss: 146.1754 - val_loss: 194.3156
Epoch 856/1000
13/13 [==============================] - 0s 8ms/step - loss: 143.1523 - val_loss: 188.4127
Epoch 857/1000
13/13 [==============================] - 0s 8ms/step - loss: 146.3553 - val_loss: 186.3149
Epoch 858/1000
13/13 [==============================] - 0s 8ms/step - loss: 146.3797 - val_loss: 205.9519
Epoch 859/1000
13/13 [==============================] - 0s 8m

13/13 [==============================] - 0s 8ms/step - loss: 145.4460 - val_loss: 185.4242
Epoch 928/1000
13/13 [==============================] - 0s 8ms/step - loss: 144.2249 - val_loss: 200.2738
Epoch 929/1000
13/13 [==============================] - 0s 8ms/step - loss: 143.7084 - val_loss: 183.9710
Epoch 930/1000
13/13 [==============================] - 0s 8ms/step - loss: 143.9478 - val_loss: 184.8506
Epoch 931/1000
13/13 [==============================] - 0s 8ms/step - loss: 143.0067 - val_loss: 188.5596
Epoch 932/1000
13/13 [==============================] - 0s 8ms/step - loss: 140.3333 - val_loss: 185.2035
Epoch 933/1000
13/13 [==============================] - 0s 8ms/step - loss: 140.5742 - val_loss: 185.2616
Epoch 934/1000
13/13 [==============================] - 0s 8ms/step - loss: 141.5810 - val_loss: 188.3222
Epoch 935/1000
13/13 [==============================] - 0s 8ms/step - loss: 140.5753 - val_loss: 188.7357
Epoch 936/1000
13/13 [==============================] - 0s 8m

In [46]:
import matplotlib.pyplot as plt


In [47]:
def learningCurve(history):
    plt.figure(figsize=(10,8))
    plt.plot(history.history['loss'], linewidth=1)
    plt.plot(history.history['val_loss'], linewidth=1)
    plt.title('Model Loss over Epochs')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['training sample loss','validation sample loss'])
    plt.savefig('Learning_curve')
    plt.show
    plt.close()

In [48]:
learningCurve(history)


In [49]:
model.save_weights('modelJ2.h5')
model_json = model.to_json()
with open("modelJ2.json", "w") as json_file:
    json_file.write(model_json)

In [51]:
np.save('tcPredtwo',df.astype('float32'))

In [52]:
np.save('gitCol', df.columns)

In [349]:
df.clo

AttributeError: 'DataFrame' object has no attribute 'clo'